In [1]:

import random
import tensorflow as tf
from tensorflow import keras
from deap import base, creator, tools

# Define valid genome options
VALID_N_CONV_LAYERS = [1, 2, 3]
VALID_CONV_FILTERS = [16, 32, 64, 128]
VALID_KERNEL_SIZES = [3, 5]
VALID_N_DENSE_LAYERS = [1, 2]
VALID_DENSE_NEURONS = [64, 128, 256]
VALID_DROPOUT_RATES = [0.0, 0.25, 0.5]
VALID_ACTIVATIONS = ['relu', 'tanh']

# Define the DEAP creator
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Define the DEAP toolbox
toolbox = base.Toolbox()
toolbox.register("n_conv_layers", random.choice, VALID_N_CONV_LAYERS)
toolbox.register("conv_filter", random.choice, VALID_CONV_FILTERS)
toolbox.register("kernel_size", random.choice, VALID_KERNEL_SIZES)
toolbox.register("n_dense_layers", random.choice, VALID_N_DENSE_LAYERS)
toolbox.register("dense_neuron", random.choice, VALID_DENSE_NEURONS)
toolbox.register("dropout_rate", random.choice, VALID_DROPOUT_RATES)
toolbox.register("activation", random.choice, VALID_ACTIVATIONS)


In [2]:

def validate_genome(individual):
    try:
        n_conv_layers = individual[0]
        if n_conv_layers not in VALID_N_CONV_LAYERS:
            return False

        conv_filters = individual[1:1 + n_conv_layers]
        if any(f not in VALID_CONV_FILTERS for f in conv_filters):
            return False

        kernel_sizes = individual[1 + n_conv_layers:1 + 2 * n_conv_layers]
        if any(k not in VALID_KERNEL_SIZES for k in kernel_sizes):
            return False

        n_dense_layers = individual[1 + 2 * n_conv_layers]
        if n_dense_layers not in VALID_N_DENSE_LAYERS:
            return False

        dense_neurons = individual[1 + 2 * n_conv_layers + 1:1 + 2 * n_conv_layers + 1 + n_dense_layers]
        if any(n not in VALID_DENSE_NEURONS for n in dense_neurons):
            return False

        dropout_rate = individual[-2]
        if dropout_rate not in VALID_DROPOUT_RATES:
            return False

        activation = individual[-1]
        if activation not in VALID_ACTIVATIONS:
            return False

        return True
    except IndexError:
        return False


In [3]:

def create_individual():
    n_conv_layers = toolbox.n_conv_layers()
    conv_filters = [toolbox.conv_filter() for _ in range(n_conv_layers)]
    kernel_sizes = [toolbox.kernel_size() for _ in range(n_conv_layers)]
    n_dense_layers = toolbox.n_dense_layers()
    dense_neurons = [toolbox.dense_neuron() for _ in range(n_dense_layers)]
    dropout_rate = toolbox.dropout_rate()
    activation = toolbox.activation()
    return creator.Individual([n_conv_layers] + conv_filters + kernel_sizes + [n_dense_layers] + dense_neurons + [dropout_rate, activation])

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [4]:

def evaluate_cnn(individual):
    if not validate_genome(individual):
        print(f"Invalid genome detected: {individual}")
        return (0.0,)  # Assign lowest fitness

    try:
        n_conv_layers = int(individual[0])
        conv_filters = individual[1:1 + n_conv_layers]
        kernel_sizes = individual[1 + n_conv_layers:1 + 2 * n_conv_layers]
        n_dense_layers_index = 1 + 2 * n_conv_layers
        n_dense_layers = int(individual[n_dense_layers_index])
        dense_neurons = individual[n_dense_layers_index + 1:n_dense_layers_index + 1 + n_dense_layers]
        dropout_rate = individual[-2]
        activation = individual[-1]

        print(f"Evaluating individual: {individual}")
        print(f"  n_conv_layers: {n_conv_layers}")
        print(f"  conv_filters: {conv_filters}")
        print(f"  kernel_sizes: {kernel_sizes}")
        print(f"  n_dense_layers: {n_dense_layers}")
        print(f"  dense_neurons: {dense_neurons}")
        print(f"  dropout_rate: {dropout_rate}")
        print(f"  activation: {activation}")

        # Simplified model evaluation placeholder
        val_acc = random.uniform(0.3, 0.9)  # Mock accuracy
        return (val_acc,)
    except Exception as e:
        print(f"Error during evaluation: {e}")
        return (0.0,)  # Assign lowest fitness

toolbox.register("evaluate", evaluate_cnn)


In [5]:

def mutate_individual(individual):
    gene_to_mutate = random.randint(0, len(individual) - 1)

    if gene_to_mutate == 0:
        individual[gene_to_mutate] = random.choice(VALID_N_CONV_LAYERS)
    elif 1 <= gene_to_mutate <= individual[0]:
        individual[gene_to_mutate] = random.choice(VALID_CONV_FILTERS)
    elif individual[0] < gene_to_mutate <= 2 * individual[0]:
        individual[gene_to_mutate] = random.choice(VALID_KERNEL_SIZES)
    elif gene_to_mutate == 2 * individual[0] + 1:
        individual[gene_to_mutate] = random.choice(VALID_N_DENSE_LAYERS)
    elif gene_to_mutate > 2 * individual[0] + 1 and gene_to_mutate < len(individual) - 2:
        individual[gene_to_mutate] = random.choice(VALID_DENSE_NEURONS)
    elif gene_to_mutate == len(individual) - 2:
        individual[gene_to_mutate] = random.choice(VALID_DROPOUT_RATES)
    elif gene_to_mutate == len(individual) - 1:
        individual[gene_to_mutate] = random.choice(VALID_ACTIVATIONS)
    
    return individual,

toolbox.register("mutate", mutate_individual)


In [6]:

def main():
    random.seed(42)
    population = toolbox.population(n=10)
    NGEN = 5
    CXPB, MUTPB = 0.5, 0.2

    for gen in range(1, NGEN + 1):
        print(f"-- Generation {gen} --")
        
        invalid_ind = [ind for ind in population if not ind.fitness.valid]
        print(f"Evaluating {len(invalid_ind)} individuals...")
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Select the next generation
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                tools.cxTwoPoint(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Replace the old population
        population[:] = offspring

        # Gather and display statistics
        fits = [ind.fitness.values[0] for ind in population]
        print(f"  Best fitness: {max(fits):.4f}")
        print(f"  Avg fitness: {sum(fits)/len(fits):.4f}")

if __name__ == "__main__":
    main()


-- Generation 1 --
Evaluating 10 individuals...
Evaluating individual: [3, 16, 16, 64, 3, 3, 3, 1, 256, 0.5, 'relu']
  n_conv_layers: 3
  conv_filters: [16, 16, 64]
  kernel_sizes: [3, 3, 3]
  n_dense_layers: 1
  dense_neurons: [256]
  dropout_rate: 0.5
  activation: relu
Evaluating individual: [3, 128, 16, 16, 3, 3, 3, 1, 256, 0.0, 'tanh']
  n_conv_layers: 3
  conv_filters: [128, 16, 16]
  kernel_sizes: [3, 3, 3]
  n_dense_layers: 1
  dense_neurons: [256]
  dropout_rate: 0.0
  activation: tanh
Evaluating individual: [1, 128, 5, 1, 64, 0.5, 'tanh']
  n_conv_layers: 1
  conv_filters: [128]
  kernel_sizes: [5]
  n_dense_layers: 1
  dense_neurons: [64]
  dropout_rate: 0.5
  activation: tanh
Evaluating individual: [2, 64, 32, 3, 5, 1, 64, 0.25, 'relu']
  n_conv_layers: 2
  conv_filters: [64, 32]
  kernel_sizes: [3, 5]
  n_dense_layers: 1
  dense_neurons: [64]
  dropout_rate: 0.25
  activation: relu
Evaluating individual: [2, 64, 64, 3, 5, 1, 128, 0.0, 'tanh']
  n_conv_layers: 2
  conv_filt

AttributeError: 'Toolbox' object has no attribute 'select'